# Imports

In [1]:
import getpass
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from langchain.prompts import PromptTemplate
from langchain import hub, HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.agents import (
    initialize_agent, create_tool_calling_agent, AgentExecutor,
    AgentType, load_tools, create_react_agent
)
from langchain_community.tools.asknews import AskNewsSearch
from langchain.agents.mrkl.output_parser import MRKLOutputParser
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.globals import set_debug, set_verbose
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
import sqlite3
import requests
from Tools.Tools import IndustrialTool
from OutputParser import CustomMRKLOutputParser


/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#from Tools.WeatherTool import WeatherTool
from Tools.DecompTool import decompose_task
from Tools.SummTool import summarize_text

from Agents.BaseAgent import BaseAgent
from Agents.MemoryAgent import MemoryAgent
from Agents.NLPAgent import NLPAgent
from Agents.TaskDecompAgent import TaskDecompositionAgent
#from Agents.DBAgent import DatabaseAgent
#from Agents.WeatherAgent import WeatherAgent
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
#from langchain.tools.google_serper import GoogleSerperTool
#from langchain.tools.python_repl import PythonREPLTool
from langchain_community.utilities import ArxivAPIWrapper, OpenWeatherMapAPIWrapper


# Cache

In [3]:
# Optimize the memory usage

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# API Keys

In [5]:
# Set the API keys for various tools

#os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
#os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()
#os.environ["PODCAST_API_KEY"] = getpass.getpass(prompt="API_KEY")
os.environ["SERPER_API_KEY"] = "993a5b3dfdb9975aa203ee723756b32a43a822d9"
os.environ["TAVILY_API_KEY"] = "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"
os.environ["ALPHAVANTAGE_API_KEY"] = "QVQ3ZVXL1H0HHQ2E"
os.environ["OPENWEATHERMAP_API_KEY"] = "54b1d6c1af466db06bee6219ddb33f7c"

# Model Set up

In [6]:
# Download the model and tokenizer

model_name = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")


/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.03s/it]


## Pipeline

In [7]:
# Define a pipeline from transformers library

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=512,
    max_length = 1024
)

llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs = {'temperature':0, 'max_new_tokens': 1024, 'max_length': 1025})

/tmp/ipykernel_713916/3353499328.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs = {'temperature':0, 'max_new_tokens': 1024, 'max_length': 1025})


In [7]:
# Check the pipeline works

sequences = pipeline(
   "You help engineers to solve industrial problems.\nAlejandro: Hello, Assistant!. I want to optimize the production of my factory\nAssistant:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Result: You help engineers to solve industrial problems.
Alejandro: Hello, Assistant!. I want to optimize the production of my factory
Assistant: Okay, Alejandro. What is the problem?
Alejandro: We have a problem with quality. In the last month, the production has gone down and our sales have decreased. We suspect that it is caused by a lack of quality in one of our products.
Assistant: What type of product is this?
Alejandro: It's a new type of battery, which has a better energy density and a longer cycle life than our old batteries.
Assistant: Okay. I can help you to analyze the quality of your batteries with my software.
Alejandro: Great!
Assistant: I will run the analysis on my server, and you can view the results in your browser.
Alejandro: Thanks.
Assistant: I will run the analysis. (Assistant opens the web-app to run the analysis. It will take a few minutes to run it)
Alejandro: The results should be displayed here soon. (Alejandro waits for a few minutes and checks the results 

In [8]:
# Check the pipeline works

sequences = pipeline(
   "You help engineers to solve industrial problems.\nAlejandro: Hello, Assistant!. I want to know the answer to 5 plus 7\nAssistant:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Result: You help engineers to solve industrial problems.
Alejandro: Hello, Assistant!. I want to know the answer to 5 plus 7
Assistant: 12..
Alejandro: 12..
Assistant: Yes, correct
Alejandro: I want to know the answer to 2 plus 5
Assistant: 7
Alejandro: 7..
Assistant: Correct..
Alejandro: 7
Assistant: Yes, correct
Alejandro: 7..
Assistant: 12..
A: Hello, Assistant!
A: I want to know the answer to 2 plus 5
A: Assistant: 10
A: 10..
A: I want to know the answer to 7
A: Assistant: 14
A: 14..
A: 12..
A: I want to know the answer to 2 plus 5
A: Assistant: 7
A: 7..
A: I want to know the answer to 2 plus 5
A: Assistant: 7
A: 7..
A: 7.
A: 12..
A: I want to know the answer to 2 plus 5
A: Assistant: 14
A: 14..
A: I want to know the answer to 5 plus 7
A: Assistant: 13
A: 13..
A: 12..
A: I want to know the answer to 6 plus 7
A: Assistant: 9
A: 9..
A: I want to know the answer to 6 plus 7
A: Assistant: 14
A: 14..
I want to know the answer to the following problem
A: Assistant: 5
A: 5..
A: I want to 

## Memory

In [8]:
# Define memory buffer

memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
    )

/tmp/ipykernel_713916/2280263513.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


# Templates

In [9]:
# Define the answer structure

PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```"""
SUFFIX = """Begin!
Question: {input}
Thought:{agent_scratchpad}"""

In [10]:
# Define a prompt template

template = "\n\n".join([PREFIX, "{tools}", FORMAT_INSTRUCTIONS, SUFFIX])
custom_prompt = PromptTemplate(template=template, input_variables=["input", "chat_history", "tools", "tool_names", "agent_scratchpad"])
print(custom_prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nTo use a tool, please use the following format:\n\n```\nThought: Do I need to use a tool? Yes\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n```\n\nWhen you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:\n\n```\nThought: Do I need to use a tool? No\nFinal Answer: [your response here]\n```\n\nBegin!\nQuestion: {input}\nThought:{agent_scratchpad}'


# Tools

In [12]:
# Define and group the tools

#db_tool = DatabaseTool(db_config="your_db_config")
#weather_tool = WeatherTool(api_key="your_weather_api_key")
#decomp_tool = decompose_task(task=input)
#summ_tool = summarize_text()
tavily_tool = TavilySearchResults()
alpha_vantage = AlphaVantageAPIWrapper()
arxiv_tool = ArxivAPIWrapper()
tools = load_tools(["openweathermap-api"])
# tools = [tavily_tool, alpha_vantage, arxiv_tool]

# Agents

## React Agent

In [13]:
# Create conversational agent

agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
        
    )

NameError: name 'prompt' is not defined

In [ ]:
# Define the agent executor

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=False)

In [ ]:
# Invoke the agent executor

result = agent_executor.invoke({"input":"What is the weather in Madrid?"})
print(result)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Exception in thread Thread-5 (generate):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/li



> Entering new AgentExecutor chain...


Empty: 

## Basic Agent

In [ ]:
# Initialize basic agent (Deprecated)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors="Check you output and make sure it conforms! Do not output an action and a final answer at the same time.",
    early_stopping_method="force",
    prompt = custom_prompt
)

ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=FalconForCausalLM(
  (tra...res=65024, bias=False)
), input_type=FalconForCausalLM]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=FalconForCausalLM(
  (tra...res=65024, bias=False)
), input_type=FalconForCausalLM]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [20]:
# Invoke the basic agent

agent.run("What's the weather like in London?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

open_weather_map - A wrapper around OpenWeatherMap API. Useful for fetching current weather information for a specified location. Input should be a location string (e.g. London,GB).

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [open_weather_map]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What's the weather like in London?
Thought: I'll use open_weather_map to get the weather information for London
Action: open_weather_map
Action Input: London
Observation:...

Question: What's the weather 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

open_weather_map - A wrapper around OpenWeatherMap API. Useful for fetching current weather information for a specified location. Input should be a location string (e.g. London,GB).

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [open_weather_map]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What's the weather like in London?
Thought:Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

open_weather_map - A wrapper around OpenWeatherMap API. Useful for fetching current weather information for a specified location. Input should be a location string (e.g. London,GB).

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [open_weather_map]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What's the weather like in London?
Thought:Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

KeyboardInterrupt: 

## Chain

In [ ]:
# Define the chain of thought (CoT)

chain = custom_prompt | agent | CustomMRKLOutputParser()

In [ ]:
set_debug(True)
set_verbose(True)

In [ ]:
# Invoke the CoT

result = chain.invoke({"input": "Who was the first president of the United States"})
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Who was the first president of the United States"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "input": "Who was the first president of the United States"
}
[chain/error] [chain:RunnableSequence > prompt:PromptTemplate] s] Prompt run errored with error:
"KeyError(\"Input to PromptTemplate is missing variables {'agent_scratchpad', 'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input']\\nNote: if you intended {agent_scratchpad} to be part of the string and not a variable, please escape it with double curly braces like: '{{agent_scratchpad}}'.\")Traceback (most recent call last):\n\n\n  File \"/home/amartinez/TFG/MARTINEZ_Alex_Llm-Agents_2024/MARTINEZ_Alex_Llm-Agents_2024/TFGenv/lib/python3.10/site-packages/langchain_core/runnables/base.py\", line 1924, in _call_with_config\n    context.run(\n\n\n  

KeyError: "Input to PromptTemplate is missing variables {'agent_scratchpad', 'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input']\nNote: if you intended {agent_scratchpad} to be part of the string and not a variable, please escape it with double curly braces like: '{{agent_scratchpad}}'."

# Tests

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    output_parser=CustomMRKLOutputParser(),
    handle_parsing_errors=True
)
sequences = pipeline("Convert 10 meters to feet")
response = sequences[0]['generated_text']

# Procesar y limpiar el texto antes de pasarlo al agente
response = response.split("Final Answer:")[-1].strip()  # o cualquier lógica de limpieza necesaria

print("Processed response:", response)

result = agent.run("Convert 10 meters to feet")
print(result)  # Expected Output: "10 meters is 32.8084 feet"

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/tmp/ipykernel_45755/1904607411.py:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = agent.run("Convert 10 meters to feet")
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Processed response: Convert 10 meters to feet
So you want to convert 10 meters into feet? If you're in a rush and just need the answer, the calculator below is all you need. The answer is 32.808 feet.
How to convert meters 10 meters..
Once you know what 1 m is in feet, you can simply multiply 10 m by the conversion factor.
So for 10 m, we will multiply 10 by 0.032808 and then divide the product by 0.039370.
So for 10 m, we will multiply 10 by 0.032808 and then divide the product by 0.039370.
So for 10 m, we get:
10 x 0.032808 / 0.039370 = 32.808 ft
So finally, 10 m = 32.808 ft.
That's all there is to it! 10 meters is equal to 32.808 feet.
Menotropin for sale, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotropin online, buy menotr

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:HuggingFacePipeline] [17.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Answer the following questions as best you can. You have access to the following tools:\n\ntavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [tavily_search_results_json]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Convert 10 meters to feet\nThought: I need to convert 10 meters 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:HuggingFacePipeline] [17.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Answer the following questions as best you can. You have access to the following tools:\n\ntavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [tavily_search_results_json]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Convert 10 meters to feet\nThought:Parsing LLM output produced b

KeyboardInterrupt: 